In [1]:
# 모델 구축
# 입력값 토큰화, 임베딩
# 훈련을 정의하고 학습
# 평가
# 예측하기 위한 데이터를 토큰화, 임베딩후 predict
# 라벨 출력

In [2]:
# import tensorflow as tf
# from tensorflow.keras import Model, layers


# # 인코더 입력 정의: 영문 단어가 입력, 각 단어는 4자 길이, 모든 문자의 수는 171개
# enc_input = layers.Input(shape=(4, 171)) 

# # 인코더 LSTM 정의: 모든 타임스텝의 출력을 반환하도록 설정
# enc_output, state_h, state_c = layers.LSTM(128, return_sequences=True, return_state=True)(enc_input)

# # 디코더 LSTM 정의: 모든 시퀀스의 출력을 반환하도록 설정
# dec_input = layers.Input(shape=(3, 171)) # 한글 단어는 2자 길이 + <Start> 토큰, 모든 문자의 수는 171개
# dec_lstm_output, _, _ = layers.LSTM(128, return_sequences=True, return_state=True)(dec_input, initial_state=[state_h, state_c])


# class SimpleAttention(layers.Layer):
#     def __init__(self):
#         super(SimpleAttention, self).__init__()

#     def call(self, encoder_outputs, decoder_output):
#         # encoder_outputs: (batch_size, enc_timesteps, units)
#         # decoder_output: (batch_size, 1, units)
#         # 어텐션 스코어 계산
#         score = tf.matmul(decoder_output, encoder_outputs, transpose_b=True)
#         # 어텐션 가중치 계산(어텐션 분포 계산)
#         attention_weights = tf.nn.softmax(score, axis=-1)
#         # 컨텍스트 벡터 계산(어텐션 벡터 계산)
#         context_vector = tf.matmul(attention_weights, encoder_outputs)
#         # 연결
#         context_vectors = []
#         context_vectors.append(context_vector)
#         for t in range(decoder_output.shape[1]):
#             context_vector = layers.Lambda(lambda x: attention_layer([encoder_outputs, tf.expand_dims(x[:, t, :], 1)]))(decoder_output)
#             context_vectors.append(context_vectors)

#         # 컨텍스트 벡터를 시퀀스 형태로 결합
#         context_vectors = layers.Lambda(lambda x: tf.concat(x, axis=1))(context_vectors)
        
#         # 디코더 출력과 어텐션 컨텍스트 벡터 결합
#         decoder_combined_context = layers.Concatenate(axis=-1)([dec_lstm_output, context_vectors])

#         return decoder_combined_context


# # 어텐션 메커니즘 적용
# decoder_combined_context = SimpleAttention()(enc_output, dec_lstm_output)

# # 디코더 출력층 정의: 출력 크기는 모든 문자의 수인 171, softmax 활성화 함수를 사용
# output = layers.TimeDistributed(layers.Dense(171, activation='softmax'))(decoder_combined_context)

# # 모델 정의: 인코더 입력(enc_input)과 디코더 입력(dec_input)을 모델의 입력으로, 디코더 출력을 모델의 출력으로 설정
# model = Model(inputs=[enc_input, dec_input], outputs=[output])

# # 모델 요약 출력
# model.summary()


In [1]:
import tensorflow as tf
from tensorflow.keras import Model, layers

class SimpleAttention(layers.Layer):
    def __init__(self):
        super(SimpleAttention, self).__init__()

    def call(self, inputs):
        encoder_outputs, decoder_output = inputs
        # encoder_outputs: (batch_size, enc_timesteps, units)
        # decoder_output: (batch_size, 1, units)
        # 어텐션 스코어 계산 (batch_size, 1, enc_timesteps)
        score = tf.matmul(decoder_output, encoder_outputs, transpose_b=True)
        # 어텐션 가중치 계산 (batch_size, 1, enc_timesteps)
        attention_weights = tf.nn.softmax(score, axis=-1)
        # 컨텍스트 벡터 계산 (batch_size, 1, units)
        context_vector = tf.matmul(attention_weights, encoder_outputs)
        return context_vector

# 인코더 입력 정의: 영문 단어가 입력, 각 단어는 4자 길이, 모든 문자의 수는 171개
enc_input = layers.Input(shape=(4, 171)) 

# 인코더 LSTM 정의: 모든 타임스텝의 출력을 반환하도록 설정
enc_output, state_h, state_c = layers.LSTM(128, return_sequences=True, return_state=True)(enc_input)

# 디코더 LSTM 정의: 모든 시퀀스의 출력을 반환하도록 설정
dec_input = layers.Input(shape=(3, 171)) # 한글 단어는 2자 길이 + <Start> 토큰, 모든 문자의 수는 171개
dec_lstm_output, _, _ = layers.LSTM(128, return_sequences=True, return_state=True)(dec_input, initial_state=[state_h, state_c])

# 어텐션 메커니즘 적용
attention_layer = SimpleAttention()

# 각 디코더 타임스텝에 대해 어텐션 적용
context_vectors = []
for t in range(dec_lstm_output.shape[1]):
    context_vector = layers.Lambda(lambda x: attention_layer([enc_output, tf.expand_dims(x[:, t, :], 1)]))(dec_lstm_output)
    context_vectors.append(context_vector)

# 컨텍스트 벡터를 시퀀스 형태로 결합
context_vectors = layers.Lambda(lambda x: tf.concat(x, axis=1))(context_vectors)

# 디코더 출력과 어텐션 컨텍스트 벡터 결합
decoder_combined_context = layers.Concatenate(axis=-1)([dec_lstm_output, context_vectors])

# 디코더 출력층 정의: 출력 크기는 모든 문자의 수인 171, softmax 활성화 함수를 사용
output = layers.TimeDistributed(layers.Dense(171, activation='softmax'))(decoder_combined_context)

# 모델 정의: 인코더 입력(enc_input)과 디코더 입력(dec_input)을 모델의 입력으로, 디코더 출력을 모델의 출력으로 설정
model = Model(inputs=[enc_input, dec_input], outputs=[output])

# 모델 요약 출력
model.summary()

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 더미 데이터 생성
import numpy as np
encoder_input_data = np.random.random((100, 4, 171))
decoder_input_data = np.random.random((100, 3, 171))
decoder_output_data = np.random.random((100, 3, 171))

# 모델 학습
model.fit([encoder_input_data, decoder_input_data], decoder_output_data, epochs=1)


TypeError: Could not build a TypeSpec for KerasTensor(type_spec=TensorSpec(shape=(None, 1, 128), dtype=tf.float32, name=None), name='simple_attention/MatMul_1:0', description="created by layer 'simple_attention'") of unsupported type <class 'keras.src.engine.keras_tensor.KerasTensor'>.

In [4]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam') # metrics가 없으면 훈련시 loss만 출력됨

In [5]:
# 문자배열 생성
import pandas as pd
import numpy as np
# <START>, <END>, <PAD>
arr1 = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz']
arr2 = pd.read_csv('korean.csv', header=None)
# print(arr2[0].values.tolist())
num_to_char = arr1 + arr2[0].values.tolist()
print(len(num_to_char))
char_to_num = {char:i for i, char in enumerate(num_to_char)}
# print(char_to_num)

171


In [6]:
# 학습용 단어셋 불러오기
raw = pd.read_csv('translate.csv', header=None)
eng_kor = raw.values.tolist()
print(len(eng_kor)) # 학습할 전체 단어 수 110개

110


In [7]:
# 단어를 숫자 배열로 변환
temp_eng = 'love'
temp_eng_n = [char_to_num[c] for c in temp_eng]
print(temp_eng_n)
temp_kor = '사랑'
np.eye(171)[temp_eng_n].shape

[14, 17, 24, 7]


(4, 171)

In [8]:
# 단어를 원-핫 인코딩된 배열로 변환
def encode(eng_kor):
    enc_in = []
    dec_in = []
    rnn_out = [] # decoder output
    for seq in eng_kor:
        eng = [char_to_num[c] for c in seq[0]]
        enc_in.append(np.eye(171)[eng])

        kor = [char_to_num[c] for c in ('S'+seq[1])]
        dec_in.append(np.eye(171)[kor])

        target = [char_to_num[c] for c in (seq[1] + 'E')]
        rnn_out.append(target)

    enc_in = np.array(enc_in)
    dec_in = np.array(dec_in)
    rnn_out = np.array(rnn_out)
    rnn_out = np.expand_dims(rnn_out, axis=2)
    return enc_in, dec_in, rnn_out

In [9]:
sample = [['word', '단어']]
encode(sample)[2]

array([[[ 61],
        [114],
        [  1]]])

In [10]:
X_enc, X_dec, y_rnn = encode(eng_kor)

In [11]:
%%time
model.fit([X_enc, X_dec], y_rnn, epochs=500)

Epoch 1/500


ValueError: Exception encountered when calling Concatenate.call().

[1mA `Concatenate` layer should be called on a list of inputs. Received: input_shape=[TensorShape([None, 3, 128]), (None, 3, 128)][0m

Arguments received by Concatenate.call():
  • args=(['tf.Tensor(shape=(None, 3, 128), dtype=float32)', '<KerasTensor shape=(None, 3, 128), dtype=float32, sparse=False, name=keras_tensor_31>'],)
  • kwargs=<class 'inspect._empty'>

In [12]:
enc_in, dec_in, _ = encode([['tall', 'PP']])
# print(enc_in)
pred = model.predict([enc_in, dec_in])
# print(pred.shape)
word = np.argmax(pred[0], axis=-1)
# print(word)
num_to_char[word[0]], num_to_char[word[1]]

ValueError: Exception encountered when calling Concatenate.call().

[1mA `Concatenate` layer should be called on a list of inputs. Received: input_shape=[TensorShape([1, 3, 128]), (None, 3, 128)][0m

Arguments received by Concatenate.call():
  • args=(['tf.Tensor(shape=(1, 3, 128), dtype=float32)', '<KerasTensor shape=(None, 3, 128), dtype=float32, sparse=False, name=keras_tensor_39>'],)
  • kwargs=<class 'inspect._empty'>

In [ ]:
from numpy.random import randint
pick = randint(0, len(eng_kor), 5)
choose = [ [eng_kor[i][0], 'PP'] for i in pick]
print(choose)

In [ ]:
enc_in, dec_in, _ = encode(choose)
pred = model.predict([enc_in, dec_in])
print(pred.shape)

In [ ]:
for i in range(len(choose)):
    eng = choose[i][0]
    word = np.argmax(pred[i], axis=-1)
    kor = ''
    for j in range(2):
        kor = kor + num_to_char[word[j]]
    print(eng, kor)